# Assignment 2 Key

In [1]:
import pandas as pd

In [2]:
import numpy as np

#### First let's load the dataset into dataframe df

In [3]:
df = pd.read_csv("train_set.csv")

/Users/mindabangong/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Take a look at the dataset first and see how many variables it has.

In [4]:
df.head(5)

,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat2,...,Var5,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount
0,1,1,3,2005,2005,K,K.78,K.78.2,D,C,...,1.008912,0.261040,0.907793,-0.077998,M,-0.23153,-0.266117,-0.272337,-0.251419,0.0
1,2,2,2,2005,2003,Q,Q.22,Q.22.3,B,C,...,1.240851,0.432987,-0.726459,0.204785,O,-0.23153,-0.266117,-0.272337,-0.251419,0.0
2,3,3,1,2005,1998,AR,AR.41,AR.41.1,B,?,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0
3,4,3,1,2006,1998,AR,AR.41,AR.41.1,B,?,...,-0.971487,-1.405797,-0.837048,-1.176858,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0
4,5,3,2,2005,2001,D,D.20,D.20.0,J,C,...,0.812656,2.112691,1.534462,2.347260,F,-0.23153,-0.266117,-0.272337,-0.251419,0.0


###    1. (1 pt.) Convert the numeric claim_amount target to a new binary variable that will be used as a target for logistic regression. Any claim_amount > 0 should be considered a claim event.

#### We converted the numeric claim_amount target to a new binary variable Claim_Event

In [5]:
df['Claim_Event'] = np.where(df['Claim_Amount']>0.0, 1, 0)

#### Here we checked if we successfully converted the variable to a new binary variable.

In [6]:
df[df.Claim_Amount!=0.0].head(5)

,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat2,...,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount,Claim_Event
47,48,16,3,2006,2005,X,X.38,X.38.0,C,C,...,-0.275857,-1.021362,-0.188486,N,-0.23153,2.783616,-0.272337,-0.251419,24.205590,1
101,102,39,1,2006,2001,BH,BH.18,BH.18.5,B,C,...,-0.650165,-0.861623,-0.527934,B,-0.23153,2.783616,2.715490,-0.251419,2.624877,1
154,155,76,5,2005,2001,AJ,AJ.48,AJ.48.1,D,C,...,1.162886,1.374723,0.452562,H,-0.23153,1.563723,-0.272337,2.404670,7.036577,1
211,212,103,1,2005,1995,AR,AR.41,AR.41.1,B,?,...,-1.405797,-0.837048,-1.193267,H,-0.23153,-0.266117,5.105752,-0.251419,8.432544,1
351,352,179,1,2005,1995,AJ,AJ.76,AJ.76.2,D,?,...,1.360567,1.423873,-0.479145,M,-0.23153,-0.266117,-0.272337,-0.251419,89.927240,1


In [7]:
len(df)

13184290

### 2. (1 pt.) Over- or under-sample the data to create a more balanced target distribution for training your logistic regression model.

#### The dataset is total imbalanced. For the Claim_Event, we have too many 0s more than 1s in the data. And since this dataset would become too large if we oversample it, we use the following codes to undersample the data and make sure that we have the same number of 0 and 1 for the Claim_Event.

In [8]:
df.groupby('Claim_Event').size()

Claim_Event
0    13088685
1       95605
dtype: int64

#### Separate majority and minority classes.

In [9]:
df_majority = df[df.Claim_Event==0]
df_minority = df[df.Claim_Event==1]

In [10]:
from sklearn.utils import resample

In [11]:
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,
                                 n_samples=95605, 
                                 random_state=123) 

#### Combine minority class with downsampled majority class

In [12]:
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

#### There are a lot of "?"  and null values in the dataset. Here we converted all the "?" to null values.

In [13]:
df_downsampled = df_downsampled.replace('?',np.nan)

In [14]:
len(df_downsampled)

191210

### 3. (1 pt.) Partition your data appropriately. Create a 30% test partition along with other partitions you may need.

#### We checked how many missing values we still have and where these missing values located.

In [15]:
df_downsampled.isnull().sum()

Row_ID                 0
Household_ID           0
Vehicle                0
Calendar_Year          0
Model_Year             0
Blind_Make           105
Blind_Model          105
Blind_Submodel       105
Cat1                 320
Cat2               69334
Cat3                  62
Cat4               79116
Cat5               79182
Cat6                 320
Cat7              102002
Cat8                  28
Cat9                   0
Cat10                 61
Cat11                420
Cat12                373
OrdCat               115
Var1                   0
Var2                   0
Var3                   0
Var4                   0
Var5                   0
Var6                   0
Var7                   0
Var8                   0
NVCat                  0
NVVar1                 0
NVVar2                 0
NVVar3                 0
NVVar4                 0
Claim_Amount           0
Claim_Event            0
dtype: int64

#### Because we have too many missing values under the cat2, cat4, cat5, cat7, simple missing value imputation method would cause biased result. That's why we chose to drop these columns.

In [16]:
df_downsampled = df_downsampled.drop(['Cat2','Cat4','Cat5','Cat7'],axis=1)

#### We splitted the data into three parts, which is 40% for train, 30% for test and 30% for validation.

In [17]:
from sklearn.cross_validation import train_test_split

/Users/mindabangong/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [18]:
train, test = train_test_split(df_downsampled, test_size=0.3, random_state=123)
train, val = train_test_split(train, test_size=0.43, random_state=123)

### 4. (1 pt.) Impute any missing values.

#### First we checked how many missing values we have in the training set then we chose to impute the missing values with mode.

In [19]:
train.isnull().sum()

Row_ID              0
Household_ID        0
Vehicle             0
Calendar_Year       0
Model_Year          0
Blind_Make         40
Blind_Model        40
Blind_Submodel     40
Cat1              131
Cat3               20
Cat6              131
Cat8               10
Cat9                0
Cat10              31
Cat11             185
Cat12             165
OrdCat             45
Var1                0
Var2                0
Var3                0
Var4                0
Var5                0
Var6                0
Var7                0
Var8                0
NVCat               0
NVVar1              0
NVVar2              0
NVVar3              0
NVVar4              0
Claim_Amount        0
Claim_Event         0
dtype: int64

#### After checking the mode number for 'Blind_Make', 'Blind_Model' and 'Blind_Submodel' for all three subsets, we found that the result is K, K.7 and AU 58.0. AU 58.0 is clearly not from the car K. So we used the second most popular submodel k.7.3 to impute the missing values for 'Blind_Submodel'.

In [20]:
train['Blind_Make'] = train.Blind_Make.fillna(train.Blind_Make.mode().item())

In [21]:
val['Blind_Make'] = val.Blind_Make.fillna(val.Blind_Make.mode().item())
test['Blind_Make'] = test.Blind_Make.fillna(test.Blind_Make.mode().item())

/Users/mindabangong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
train['Blind_Model'] = train.Blind_Model.fillna(train.Blind_Model.mode().item())

In [23]:
val['Blind_Model'] = val.Blind_Model.fillna(val.Blind_Model.mode().item())
test['Blind_Model'] = test.Blind_Model.fillna(test.Blind_Model.mode().item())

/Users/mindabangong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
train['Blind_Submodel'] = train.Blind_Submodel.fillna('K.7.3')

In [25]:
val['Blind_Submodel'] = val.Blind_Submodel.fillna('K.7.3')
test['Blind_Submodel'] = test.Blind_Submodel.fillna('K.7.3')

/Users/mindabangong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
len(train[train.isnull().any(axis=1)])

422

In [27]:
len(test[test.isnull().any(axis=1)])

289

In [28]:
len(val[val.isnull().any(axis=1)])

293

#### Because we have different data types under the OrdCat variable, including integer and string. We converted all of them into string type.

In [29]:
train['OrdCat'] = train['OrdCat'].astype(str)

In [30]:
test['OrdCat'] = test['OrdCat'].astype(str)
val['OrdCat'] = val['OrdCat'].astype(str)

/Users/mindabangong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


#### We used the same method to fill in the missing values for the following categorical variables with mode in the training set, testing set and the validation set.

In [31]:
train['Cat1'] = train.Cat1.fillna(train.Cat1.mode().item())

In [32]:
train['Cat3'] = train.Cat3.fillna(train.Cat3.mode().item())
train['Cat6'] = train.Cat6.fillna(train.Cat6.mode().item())
train['Cat8'] = train.Cat8.fillna(train.Cat8.mode().item())
train['Cat9'] = train.Cat9.fillna(train.Cat9.mode().item())
train['Cat10'] = train.Cat10.fillna(train.Cat10.mode().item())
train['Cat11'] = train.Cat11.fillna(train.Cat11.mode().item())
train['Cat12'] = train.Cat12.fillna(train.Cat12.mode().item())

#### We converted OrdCat by "astype(str)", for some reasons the "NaN" is considered as a string, thus the "fillna()" function does not work anymore. We used replace function to impute missing values with mode instead.

In [33]:
train['OrdCat'] = train.OrdCat.fillna(train.OrdCat.mode().item())

In [34]:
test['Cat1'] = test.Cat1.fillna(test.Cat1.mode().item())
test['Cat3'] = test.Cat3.fillna(test.Cat3.mode().item())
test['Cat6'] = test.Cat6.fillna(test.Cat6.mode().item())
test['Cat8'] = test.Cat8.fillna(test.Cat8.mode().item())
test['Cat10'] = test.Cat10.fillna(test.Cat10.mode().item())
test['Cat11'] = test.Cat11.fillna(test.Cat11.mode().item())
test['Cat12'] = test.Cat12.fillna(test.Cat12.mode().item())
test['OrdCat'] = test.OrdCat.fillna(test.OrdCat.mode().item())

/Users/mindabangong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/mindabangong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/mindabangong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [35]:
test.isnull().sum()

Row_ID            0
Household_ID      0
Vehicle           0
Calendar_Year     0
Model_Year        0
Blind_Make        0
Blind_Model       0
Blind_Submodel    0
Cat1              0
Cat3              0
Cat6              0
Cat8              0
Cat9              0
Cat10             0
Cat11             0
Cat12             0
OrdCat            0
Var1              0
Var2              0
Var3              0
Var4              0
Var5              0
Var6              0
Var7              0
Var8              0
NVCat             0
NVVar1            0
NVVar2            0
NVVar3            0
NVVar4            0
Claim_Amount      0
Claim_Event       0
dtype: int64

In [36]:
val['Cat1'] = val.Cat1.fillna(val.Cat1.mode().item())
val['Cat3'] = val.Cat3.fillna(val.Cat3.mode().item())
val['Cat6'] = val.Cat6.fillna(val.Cat6.mode().item())
val['Cat8'] = val.Cat8.fillna(val.Cat8.mode().item())
val['Cat10'] = val.Cat10.fillna(val.Cat10.mode().item())
val['Cat11'] = val.Cat11.fillna(val.Cat11.mode().item())
val['Cat12'] = val.Cat12.fillna(val.Cat12.mode().item())
val['OrdCat'] = val.OrdCat.fillna(val.OrdCat.mode().item())

In [37]:
val.isnull().sum()

Row_ID            0
Household_ID      0
Vehicle           0
Calendar_Year     0
Model_Year        0
Blind_Make        0
Blind_Model       0
Blind_Submodel    0
Cat1              0
Cat3              0
Cat6              0
Cat8              0
Cat9              0
Cat10             0
Cat11             0
Cat12             0
OrdCat            0
Var1              0
Var2              0
Var3              0
Var4              0
Var5              0
Var6              0
Var7              0
Var8              0
NVCat             0
NVVar1            0
NVVar2            0
NVVar3            0
NVVar4            0
Claim_Amount      0
Claim_Event       0
dtype: int64

### 5. (2 pts.) Create numeric features for the blind_make, blind_model, and blind_submodel high cardinality categorical variables and use them in your model.

In [38]:
from numpy.random import uniform

In [39]:
train.columns

Index(['Row_ID', 'Household_ID', 'Vehicle', 'Calendar_Year', 'Model_Year',
       'Blind_Make', 'Blind_Model', 'Blind_Submodel', 'Cat1', 'Cat3', 'Cat6',
       'Cat8', 'Cat9', 'Cat10', 'Cat11', 'Cat12', 'OrdCat', 'Var1', 'Var2',
       'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'NVCat', 'NVVar1',
       'NVVar2', 'NVVar3', 'NVVar4', 'Claim_Amount', 'Claim_Event'],
      dtype='object')

####  We created numeric features for the blind_make, blind_model, and blind_submodel variables.

In [40]:
for col_name in train.columns[5:8]:
    new_col_name = col_name + '_encode'
    # create a dictionary of original categorical value:event rate for that value
    row_val_dict = {}
    for level in train[col_name].unique():
        row_val_dict[level] = train[train[col_name] == level]['Claim_Event'].mean()
    # apply the transform from the dictionary on all rows in the column
    train[new_col_name] = train[col_name].apply(lambda i: row_val_dict[i] + uniform(low=-0.05, high=0.05))

In [41]:
for col_name in val.columns[5:8]:
    new_col_name = col_name + '_encode'
    # create a dictionary of original categorical value:event rate for that value
    row_val_dict = {}
    for level in val[col_name].unique():
        row_val_dict[level] = val[val[col_name] == level]['Claim_Event'].mean()
    # apply the transform from the dictionary on all rows in the column
    val[new_col_name] = val[col_name].apply(lambda i: row_val_dict[i] + uniform(low=-0.05, high=0.05))

In [42]:
for col_name in test.columns[5:8]:
    new_col_name = col_name + '_encode'
    # create a dictionary of original categorical value:event rate for that value
    row_val_dict = {}
    for level in test[col_name].unique():
        row_val_dict[level] = test[test[col_name] == level]['Claim_Event'].mean()
    # apply the transform from the dictionary on all rows in the column
    test[new_col_name] = test[col_name].apply(lambda i: row_val_dict[i] + uniform(low=-0.05, high=0.05))

/Users/mindabangong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [43]:
train.head(5)

,Row_ID,Household_ID,Vehicle,Calendar_Year,Model_Year,Blind_Make,Blind_Model,Blind_Submodel,Cat1,Cat3,...,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Amount,Claim_Event,Blind_Make_encode,Blind_Model_encode,Blind_Submodel_encode
11579586,11579587,6839840,1,2005,1997,K,K.7,K.7.0,B,A,...,M,-0.23153,-0.266117,-0.272337,-0.251419,0.00000,0,0.471561,0.533590,0.539134
13181757,13181758,7540085,1,2005,2003,AU,AU.40,AU.40.0,I,F,...,O,-0.23153,-0.266117,-0.272337,-0.251419,12.38141,1,0.581584,0.510680,0.507141
8144269,8144270,5202275,1,2007,1999,K,K.7,K.7.1,B,A,...,M,-0.23153,-0.266117,-0.272337,-0.251419,0.00000,0,0.543132,0.561698,0.562724
10928815,10928816,6622402,1,2006,1993,K,K.51,K.51.0,D,C,...,M,-0.23153,-0.266117,-0.272337,-0.251419,0.00000,0,0.528122,0.410964,0.480145
2006297,2006298,1203346,2,2007,1990,AJ,AJ.76,AJ.76.2,D,C,...,M,-0.23153,-0.266117,-0.272337,-0.251419,0.00000,0,0.513446,0.496811,0.463755


### 6. (1 pt.) Train a logistic regression model with variable selection based on minimizing validation error or maximizing another suitable validation accuracy measure.

#### Because theoretically Row_ID, House_ID and Vehicle have nothing to do with the result of Claim_Event, we dropped all the useless variables out of the dataset.

In [44]:
train = train.drop(['Row_ID','Household_ID','Vehicle','Blind_Make','Blind_Model','Blind_Submodel','Claim_Amount'],axis=1)

In [45]:
val = val.drop(['Row_ID','Household_ID','Vehicle','Blind_Make','Blind_Model','Blind_Submodel','Claim_Amount'],axis=1)
test = test.drop(['Row_ID','Household_ID','Vehicle','Blind_Make','Blind_Model','Blind_Submodel','Claim_Amount'],axis=1)

#### We loaded H20 to train a logistic regression model.

In [46]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [47]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,7 days 20 hours 52 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,"14 days, 22 hours and 57 minutes"
H2O cluster name:,H2O_from_python_mindabangong_k0k5xv
H2O cluster total nodes:,1
H2O cluster free memory:,590 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [48]:
trainhf = h2o.H2OFrame(train)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [49]:
valhf = h2o.H2OFrame(val)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [50]:
testhf = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


#### We set target and categorical variables to factor for logisitic regression model.

In [51]:
trainhf['OrdCat'] = trainhf['OrdCat'].asfactor()
testhf['OrdCat'] = testhf['OrdCat'].asfactor()
valhf['OrdCat'] = valhf['OrdCat'].asfactor()

In [52]:
trainhf['Calendar_Year'] = trainhf['Calendar_Year'].asfactor()
testhf['Calendar_Year'] = testhf['Calendar_Year'].asfactor()
valhf['Calendar_Year'] = valhf['Calendar_Year'].asfactor()

In [53]:
trainhf['Model_Year'] = trainhf['Model_Year'].asfactor()
testhf['Model_Year'] = testhf['Model_Year'].asfactor()
valhf['Model_Year'] = valhf['Model_Year'].asfactor()

In [54]:
y = 'Claim_Event'
X = [name for name in trainhf.columns if name not in ['C1',y]]

In [55]:
print(y)
print(X)

Claim_Event
['Calendar_Year', 'Model_Year', 'Cat1', 'Cat3', 'Cat6', 'Cat8', 'Cat9', 'Cat10', 'Cat11', 'Cat12', 'OrdCat', 'Var1', 'Var2', 'Var3', 'Var4', 'Var5', 'Var6', 'Var7', 'Var8', 'NVCat', 'NVVar1', 'NVVar2', 'NVVar3', 'NVVar4', 'Blind_Make_encode', 'Blind_Model_encode', 'Blind_Submodel_encode']


In [56]:
trainhf[y] = trainhf[y].asfactor()
testhf[y] = testhf[y].asfactor()
valhf[y] = valhf[y].asfactor()

In [57]:
trainhf.describe()

Rows:76292
Cols:28




,Calendar_Year,Model_Year,Cat1,Cat3,Cat6,Cat8,Cat9,Cat10,Cat11,Cat12,OrdCat,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,NVCat,NVVar1,NVVar2,NVVar3,NVVar4,Claim_Event,Blind_Make_encode,Blind_Model_encode,Blind_Submodel_encode
type,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,enum,real,real,real,real,real,real,real,real,enum,real,real,real,real,enum,real,real,real
mins,,,,,,,,,,,,-2.5782217999999997,-2.4069371,-2.7903352000000003,-2.4236476,-3.3503442000000003,-2.3345472999999997,-2.7784905,-1.9651488,,-0.2315299,-0.2661168,-0.27233720000000006,-0.2514189,,-0.04972123772028189,-0.049879298535897024,-0.04985531168839318
mean,,,,,,,,,,,,-0.02949798548602732,-0.096161874883343,-0.0556787600652756,-0.08478846161196461,-0.008717678603261145,-0.07175581118334816,-0.052634356451528384,-0.0863067037172967,,0.03548116248492627,0.10341345003276883,0.1100108745333718,0.0419018123800661,,0.49972869156365696,0.49981000495504774,0.4998491743482984
maxs,,,,,,,,,,,,5.1433916,7.2069383,5.5633254,6.980369099999999,3.8635409,4.5842886,3.9059711,33.457737,,6.627110000000001,8.8830814,8.691144,6.3888025,,1.025374490600591,1.0446987748099794,1.049957919687348
sigma,,,,,,,,,,,,0.9645341672012594,0.9457363074845767,1.006548220765828,0.9458107717305771,0.9759744127356131,0.9711688983097996,1.0002479524804553,0.9423646456781296,,1.065288598294371,1.1780800946125565,1.1731477240113848,1.0774940980381331,,0.0430776754686454,0.08136705794402514,0.10065186683142653
zeros,,,,,,,,,,,,12,14,131,14,131,80,131,131,,0,0,0,0,,0,0,0
missing,0,0,0,0,0,0,0,0,0,0,45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,2005,1997,B,A,B,A,B,A,E,C,2,-0.5453724,-0.9890633,-0.4531614000000001,-0.9859833,-0.15672809999999998,-0.7051409000000001,-0.8984856999999999,-0.4080388,M,-0.2315299,-0.2661168,-0.27233720000000006,-0.2514189,0,0.4715605311860616,0.5335900268978209,0.539134435593037
1,2005,2003,I,F,B,A,A,A,A,C,4,-0.4569876,0.9129624,-0.0829161,0.9929196,-0.5016624,0.6365166999999999,0.6620411,0.7040576,O,-0.2315299,-0.2661168,-0.27233720000000006,-0.2514189,1,0.581583944828677,0.5106802498947309,0.507141090944622
2,2007,1999,B,A,B,C,B,B,A,D,2,-0.5453724,-0.9890633,-0.4531614000000001,-0.9521559000000002,-0.15672809999999998,-0.6794073,-0.8984856999999999,-0.3314632,M,-0.2315299,-0.2661168,-0.27233720000000006,-0.2514189,0,0.5431322989165766,0.5616976778583459,0.5627237243481078


#### We built the logistic regression model on training dataset.

In [58]:
claim_logistic = H2OGeneralizedLinearEstimator(family='binomial',
                                         model_id='claim_logistic',
                                         solver='IRLSM',
                                         standardize=True,
                                         lambda_search=True)


claim_logistic.train(X, y, training_frame=trainhf, validation_frame=valhf)


glm Model Build progress: |███████████████████████████████████████████████| 100%


### 7. (1 pt.) Report your test AUC and give its exact interpretation.

#### We calculated the AUC score in three datasets to see if our model is good enough to fit the dataset.

In [59]:
print(claim_logistic.auc(train=True))
print(claim_logistic.auc(valid=True))
print(claim_logistic.model_performance(test_data=testhf).auc())

0.6244767149524039
0.632581533967085
0.636464642513237


#### The interpretation of AUC is: If we randomly select a true YES and a true NO, the probability that our model gives a higher probability to the true YES than to the true NO is 0.636

### 8. (2 pts.) Explain the 3 most important variables in your model in terms of the odds that a policy will have a claim associated with it.

In [60]:
# print non-zero model parameters
for name, val in claim_logistic.coef().items():
    if val != 0.0:
        print(name, ': ', val)

Intercept :  -1.9292735603738163
NVCat.L :  0.021659802953515286
NVCat.M :  -0.3512657459731928
NVCat.O :  -0.08946597502466531
OrdCat.4 :  0.0025374790873976504
Cat3.A :  0.03384613577383472
Cat3.C :  -0.0109737967737661
Cat11.A :  -0.006457662915027659
Calendar_Year.2005 :  0.030807762183225767
Var2 :  -0.004173916706301869
Var3 :  0.003964791819310119
Var5 :  0.005406015598870943
Var7 :  0.022476863820086176
Var8 :  0.0292711859428556
NVVar1 :  0.014871859924714957
NVVar2 :  0.0898931509488534
NVVar3 :  0.10406447895621516
NVVar4 :  0.017009708348955415
Blind_Model_encode :  0.5121707781818976
Blind_Submodel_encode :  3.561402451665348


In [61]:
h2o.cluster().shutdown(prompt=False)

H2O session _sid_b5fe closed.


#### The 3 most important variables are Blind_Model_encode, Blind_Submodel_encode and NVCat.M which is a level of the NVCat variable. 

* For a 1 unit increase in blind_model_encode the odds of a claim occuring change by a factor of 1.67 on average holding all other variables constant. 

* For a 1 unit increase in blind_submodel_encode the odds of a claim occuring change by a factor of 35.1 on average holding all other variables constant.

* When the NVCat = M the odds of a claim change by a factor of 0.7 on average holding all other variables constant. (There are no reference levels for penalized GLM in H2O.) 